# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [1]:
import sys
from typing import Callable

sys.stdout.write('Hello, my friend!')
original_write = sys.stdout.write

Hello, my friend!

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [2]:
from datetime import datetime

# хочу сохранить, с какими аргументами вызывается write
# далее будет видно зачем это
def save_call_history(history: list):
    """
    Decorator to save the called arguments history. (not kwargs).
    
    Args:
        history (list): variable where we want to save the history.
    """
    def decorator(f: Callable):
        def wrapper(*args, **kwargs):
            history.append((args, kwargs))
            return f(*args, **kwargs)
        return wrapper
    return decorator

call_history = []

@save_call_history(call_history)
def my_write(string_text: str):
    """
    Writes string_text prefixed with the date the function was called.
    
    Template: '[%Y-%m-%d %H:%M:%S]: {string_text}'
    """
    str_with_date = datetime.now().strftime('[%Y-%m-%d %H:%M:%S]: ') \
        + string_text
    #  В счет беру также записанные байты от добавления даты.
    #  Из условия неясно, нужно это делать или нет,
    #  Зависит от нашей цели.
    return original_write(str_with_date)

sys.stdout.write = my_write

In [3]:
print('1, 2, 3')

[2023-12-14 04:10:45]: 1, 2, 3[2023-12-14 04:10:45]: 


Дата печатаеся два раза. Вот почему:

In [4]:
call_history

[(('1, 2, 3',), {}), (('\n',), {})]

Видно, что когда мы вызываем `print`, то write вызывается два раза. С контентом и с `end`. В прошлый раз он вызвался со стандартным `end='\n'`.

Считаю, что это то поведение, которое нужно. [Тут](https://github.com/python/cpython/blob/v3.11.2/Python/bltinmodule.c#L1986), в реализации CPython, видно, что  `PyFile_WriteObject` и `PyFile_WriteString` (который на самом деле внутри вызывает WriteObject) будут вызываться для аргументов, сепараторов и end по отдельности, а эти функции внутри вызывают сишный write. То есть, важно, что не создается одна общая строка, которая выводится, а вызывается write для отдельных компонентов.

In [5]:
sys.stdout.write = original_write

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [6]:
def timed_output(f: Callable) -> Callable:
    # В нашем случае, они уже как бы объявлены, но я
    # для более общего вида тут объявляю еще раз.
    original_write = sys.stdout.write
    # Я тут добвил проверку на то, что после strip()
    # в строке что-то остается. Это только для красивого вида.
    my_write = lambda string_text: original_write(
        datetime.now().strftime('[%Y-%m-%d %H:%M:%S]: ') \
            + string_text) if string_text.strip() != '' else 0

    def wrapper(*args, **kwargs):
        sys.stdout.write = my_write
        rt = f(*args, **kwargs)
        sys.stdout.write = original_write
        return rt
    return wrapper

In [7]:
h = []
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [8]:
print_greeting("Nikita")

[2023-12-14 04:10:52]: Hello, Nikita!

Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод функции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [9]:
def redirect_output(filepath: str, mode='a'):
    """Redirect output of write to a file (filepath) openned with mode."""
    original_write = sys.stdout
    def decorator(f: Callable):
        def wrapper(*args, **kwargs):
            new_write = open(filepath, mode)
            sys.stdout = new_write
            rt = f(*args, **kwargs)
            new_write.close()
            sys.stdout = original_write
            return rt
        return wrapper
    return decorator

In [10]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()

In [11]:
%cat function_output.txt

cat: function_output.txt: No such file or directory


In [12]:
calculate()

In [13]:
%cat function_output.txt

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361 
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859 
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321 
